In [31]:
from fastai.text.all import *

In [32]:
import torch
torch.cuda.set_device(2)

In [55]:
path = untar_data(URLs.IMDB)

In [34]:
shutil.move("/.fastai/data/imdb","../data")

'../data/imdb'

In [35]:
path = "../data/imdb"

In [36]:
# move data into different folders for dataset size (default 25000 train 25000 test)
import os
import shutil
# equivalent to total dataset size of 10000, 20000, 30000, and 40000 respectively
data_sizes = [2500, 5000, 7500, 10000]
status = ['train', 'test']
category = ['pos', 'neg']
for num in data_sizes:
    for t in status:
        for name in category:
            src = f"../data/imdb/{t}/{name}"
            src_files = os.listdir(src)
            dest = f"../data/imdb{num}/{t}/{name}"
            i = 0
            for file_name in src_files:
                full_file_name = os.path.join(src, file_name)
                if os.path.isfile(full_file_name):
                    if (not os.path.isdir(dest)):
                        os.makedirs(dest)
                    shutil.copy(full_file_name, dest)
                i += 1
                if not (i < num):
                    break
            print (len([name for name in os.listdir(dest) if os.path.isfile(os.path.join(dest, name))]))

2500
2500
2500
2500
5000
5000
5000
5000
7500
7500
7500
7500
10000
10000
10000
10000


In [37]:
dls = DataBlock(
    blocks=(TextBlock.from_folder(path, max_vocab=60000),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
)

In [38]:
dls = dls.dataloaders(path)
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj in xxup nyc , seaman xxmaj michael o'hara ( orson xxmaj welles ) rescues xxmaj elsa xxmaj bannister ( rita xxmaj hayworth ) from a mugging & rape as she takes a horse & carriage through xxmaj central xxmaj park -and lives to regret it . xxmaj xxunk - haired xxmaj hayworth 's a platinum blonde in this one ; as dazzling as fresh - fallen snow -but nowhere near as pure … \n\n xxmaj to reveal any more of the convoluted plot in this seminal "" noir "" would be criminal . xxmaj it 's as deceptive as the mirrors used to cataclysmic effect in the final scenes -but the film holds far darker secrets : xxmaj from the xxup ny xxmaj times : "" childhood xxmaj shadows : xxmaj the xxmaj hidden xxmaj story xxmaj of xxmaj the xxmaj black xxmaj dahlia xxmaj murder "" by",pos
2,"xxbos xxrep 3 * xxup spoilers xxrep 3 * xxrep 3 * xxup spoilers xxrep 3 * xxmaj continued … \n\n xxmaj from here on in the whole movie collapses in on itself . xxmaj first we meet a rogue program with the indication we 're gon na get ghosts and vampires and werewolves and the like . xxmaj we get a guy with a retarded accent talking endless garbage , two ' ghosts ' that serve no real purpose and have no character what - so - ever and a bunch of henchmen . xxmaj someone 's told me they 're vampires ( straight out of xxmaj blade 2 ) , but they 're so undefined i did n't realise . \n\n xxmaj the funny accented guy with a ridiculous name suffers the same problem as the xxmaj oracle , only for far longer and far far worse .",neg
3,"xxbos xxup myra xxup breckinridge is one of those rare films that established its place in film history immediately . xxmaj praise for the film was absolutely nonexistent , even from the people involved in making it . xxmaj this film was loathed from day one . xxmaj while every now and then one will come across some maverick who will praise the film on philosophical grounds ( aggressive feminism or the courage to tackle the issue of xxunk ) , the film has not developed a cult following like some notorious flops do . xxmaj it 's not hailed as a misunderstood masterpiece like xxup scarface , or trotted out to be ridiculed as a camp classic like xxup showgirls . \n\n xxmaj undoubtedly the reason is that the film , though outrageously awful , is not lovable , or even likable . xxup myra xxup breckinridge is just",neg
4,"xxbos xxmaj my xxmaj comments for xxup vivah : - xxmaj its a charming , idealistic love story starring xxmaj shahid xxmaj kapoor and xxmaj amrita xxmaj rao . xxmaj the film takes us back to small pleasures like the bride and bridegroom 's families sleeping on the floor , playing games together , their friendly banter and mutual respect . xxmaj vivah is about the sanctity of marriage and the importance of commitment between two individuals . xxmaj yes , the central romance is naively visualized . xxmaj but the sneaked - in romantic moments between the to - be - married couple and their stubborn resistance to modern courtship games makes you crave for the idealism . xxmaj the film predictably concludes with the marriage and the groom , on the wedding night , tells his new bride who suffers from burn injur

In [39]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [40]:
learn.fit_one_cycle(1, 1e-2)
learn.save('1ephoch')

epoch,train_loss,valid_loss,accuracy,time
0,0.441856,0.369329,0.837960,01:14


Path('models/1ephoch.pth')

In [41]:
learn.freeze_to(-2)
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,accuracy,time
0,0.414833,0.359407,0.843600,01:23
1,0.393375,0.327464,0.861040,01:23
2,0.380391,0.314404,0.867080,01:23
3,0.346118,0.304185,0.870840,01:23
4,0.350588,0.304303,0.871800,01:23
5,0.358981,0.309317,0.869000,01:23


In [42]:
learn.freeze_to(-3)
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,accuracy,time
0,0.337304,0.296929,0.875320,01:55
1,0.300857,0.268777,0.889920,01:55
2,0.292113,0.247875,0.900800,01:55
3,0.265077,0.238805,0.905560,01:55
4,0.250568,0.232246,0.908240,01:55
5,0.235887,0.233259,0.907600,01:55


In [43]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-4)

epoch,train_loss,valid_loss,accuracy,time
0,0.248448,0.231008,0.909560,02:20
1,0.224172,0.211486,0.916880,02:20
2,0.209509,0.208583,0.918000,02:20
3,0.188199,0.214837,0.920200,02:20
4,0.155220,0.201252,0.924960,02:20
5,0.142180,0.212511,0.922840,02:20
6,0.114416,0.208440,0.924800,02:20
7,0.107821,0.202833,0.927640,02:20
8,0.099366,0.208126,0.925320,02:20
9,0.095493,0.205990,0.926360,02:20


In [44]:
learn.save('23ephoch')

Path('models/23ephoch.pth')

In [54]:
var = learn.show_results(max_n=10, shuffle=True)

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , is n't much different at all from the previous games ( excluding xxmaj tony xxmaj hawk 3 ) . xxmaj the only thing new that is featured in xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , is the new selection of levels , and tweaked out graphics . xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x offers a new career mode , and that is the 2x career . xxmaj the 2x career is basically xxmaj tony xxmaj hawk 1 career , because there is only about five challenges per level . xxmaj if you missed xxmaj tony xxmaj hawk 1 and 2 , i suggest that you buy xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , but if you have played the first two games , you should still",pos,pos
2,"xxbos xxmaj based on the excellent novel , xxmaj watchers by xxmaj dean xxmaj koontz , is this extremely awful motion picture that probably should n't be viewed by anyone . xxmaj not since "" the xxmaj running xxmaj man "" have i seen a book butchered so far beyond recognition . xxmaj the difference , however , is that "" the xxmaj running xxmaj man "" film was still enjoyable as an amusing action film laden down a million catch phrases . xxmaj this film ▁ xxmaj nope , nothing remotely amusing . xxmaj in fact , if you love the book , as i do , you 'll hate this bastardization even more . \n\n * * xxunk xxup spoilers * * xxmaj xxunk , xxmaj i 'm basically going to tell you the story here , almost in it 's entirety . xxmaj why ? xxmaj because",neg,neg
3,"xxbos xxmaj hearkening back to those "" good xxmaj old xxmaj days "" of 1971 , we can vividly recall when we were treated with a whole xxmaj season of xxmaj charles xxmaj chaplin at the xxmaj cinema . xxmaj that 's what the promotional guy called it when we saw him on somebody 's old talk show . ( we ca n't recall just whose it was ; either xxup merv xxup griffin or xxup woody xxup woodbury , one or the other ! ) xxmaj the guest talked about xxmaj sir xxmaj charles ' career and how his films had been out of circulation ever since the 1952 exclusion of the former "" little xxmaj tramp ' from xxmaj los xxmaj xxunk xxmaj xxunk on the grounds of his being an "" undesirable xxmaj alien "" . ( no xxmaj schultz , he 's xxup not from another",pos,pos
4,"xxbos "" empire xxmaj strikes xxmaj back "" director xxmaj irvin xxmaj kershner 's "" never xxmaj say xxmaj never xxmaj again , "" a remake of the 1965 xxmaj james xxmaj bond movie "" thunderball , "" does n't surpasses the xxmaj terence xxmaj young original , but this non - harry xxmaj saltzman & xxmaj albert xxup r. xxmaj broccoli film is well worth watching if you call yourself a 007 aficionado . xxmaj nevertheless , despite its shortage of clever gadgets and the lack of a vibrant musical score , "" never xxmaj say xxmaj never xxmaj again "" rates as an above - average , suspenseful doomsday thriller with top - flight performances by a seasoned cast including xxmaj sean xxmaj connery , xxmaj kim xxmaj basinger , xxmaj klaus xxmaj maria xxmaj brandauer , xxmaj max xxmaj von xxmaj sydow , xxmaj barbara xxmaj carrera",pos,pos
5,"xxbos xxmaj during an eight - year stretch of the 1980s when xxmaj charles xxmaj bronson made nine films , only one was released by a company o